<a href="https://colab.research.google.com/github/aviral-shrivastava/Sentimental-analysis/blob/main/Sentimental_Analysis_for_Social_Media(logistic_regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install instaloader textblob nltk transformers wordcloud matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for instaloader: filename=instaloader-4.14-py3-none-any.whl size=67805 sha256=f4de979d24e31f5e7aa2663285821cbb56ab2a3567336ec8e4a6f813f0b37fb0
  Stored in directory: /root/.cache/pip/wheels/af/c4/a8/eff3f91c3ba97d9a2b9a163e395cee000a99bc99557863db57
Successfully built instaloader


In [ ]:
! pip install kaggle

In [ ]:
! pip install GetOldTweets3

Upload Kaggle.json file


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing Twitter Sentiment Dataset

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 91% 74.0M/80.9M [00:00<00:00, 81.2MB/s]
100% 80.9M/80.9M [00:00<00:00, 96.8MB/s]


In [ ]:
#Extracting the Compressed Data

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Printing the Stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [ ]:
#Loading the Data from csv File to Pandas Dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO - 8859-1')

In [ ]:
# Checking the number of Rows and Columns
twitter_data.shape

(1599999, 6)

In [ ]:
# Printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
column_names=['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding = 'ISO - 8859-1')

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
#changing the target from 4 to 1 for ease of use
twitter_data.replace([4], [1], inplace=True)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 = Negative comments
1= Positive comments


STEMMING (reducing the word to its root word)

In [ ]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmend_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmend_content)
    return stemmed_content

In [ ]:
#twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.to_csv('twitter_data_with_stemmed_content.csv', index=False)


In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


SEPERATE THE DATA AND LABEL

In [ ]:
# prompt: make two variables x and y with stemmed content and target

X = twitter_data['text'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
 ... 'Are you ready for your MoJo Makeover? Ask me for details '
 'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur '
 'happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H ']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


SPLITTING THE DATA INTO TRAINING AND TEST

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
print(X_train)

['about to watch saw iv and drink a lil wine ' "@HaterMagazine I'm in! "
 'even though its my favourite drink i think its the vodka and coke that wipes my mind all the time  think im gonna have to find a new drink'
 ... 'is eager for Monday afternoon '
 "Hope everyone and their mother had a great day!  Can't wait to hear what the guys have in store tomorrow!"
 'I love waking up to Folgers. Too bad my voice was deeper than his. ']


In [ ]:
print(X_test)

["@mmangen - M doing fine! I haven't had much time to chat on Twitter.   Hubby is back for the summer &amp; tends to dominate my free time. "
 'at AHS may show w/  ruth kim &amp; geoffrey sanhueza '
 '@iShatara maybe it was only a bay area thang  dammit!' ...
 '@Destini41 Nevertheless Hooray! for 4700 members and have a wonderful and safe trip! '
 'Not feeling too well ' '@supersandro !!!! thank you! ']


CONVERT THE TEXTUAL DAT TO NUMERICAL DATA

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

print(X_test)

  (0, 56126)	0.1488769455924407
  (0, 75487)	0.14090004365001982
  (0, 119150)	0.24248183645807006
  (0, 165538)	0.18104096237371697
  (0, 165924)	0.37505416545454756
  (0, 197441)	0.22153830924032764
  (0, 201909)	0.09906588316358036
  (0, 204417)	0.20529869009944654
  (0, 226659)	0.1462095050939062
  (0, 230930)	0.20171242685035556
  (0, 242946)	0.24412723577494455
  (0, 256825)	0.0961353399433816
  (0, 360633)	0.39258757628101926
  (0, 370285)	0.1543215184822133
  (0, 372800)	0.08816083516534924
  (0, 397223)	0.10689068511219836
  (0, 501895)	0.1910050343105614
  (0, 514780)	0.36028752155055377
  (0, 517424)	0.0748447620172574
  (0, 526358)	0.280616950992062
  (0, 528767)	0.14519551210814186
  (0, 540515)	0.15792536311962385
  (1, 46601)	0.5297958040483101
  (1, 56126)	0.18177926727823518
  (1, 69643)	0.14611121221507706
  :	:
  (319995, 540515)	0.20109769117236578
  (319995, 568406)	0.28020395219976785
  (319996, 45246)	0.29061561010339665
  (319996, 436675)	0.2838676052428302
  (3

In [ ]:
print(X_train)

  (0, 40489)	0.24120756614816113
  (0, 528767)	0.12112707573781936
  (0, 558927)	0.3035607951325368
  (0, 461482)	0.3283935062719776
  (0, 259193)	0.48840858232447565
  (0, 57473)	0.14918233689170618
  (0, 169798)	0.3832112649811927
  (0, 317443)	0.38662946862459446
  (0, 566317)	0.4123398570437823
  (1, 230652)	0.9765050549311477
  (1, 251985)	0.21549449573925625
  (2, 528767)	0.06899951768639999
  (2, 57473)	0.08498107652774642
  (2, 169798)	0.4365892975560666
  (2, 187743)	0.16659329598173891
  (2, 524174)	0.1594579036569183
  (2, 258127)	0.283734206417887
  (2, 372800)	0.16758245532296176
  (2, 194028)	0.2506594792323131
  (2, 523182)	0.28645427621584585
  (2, 517424)	0.14227030589479395
  (2, 555192)	0.2817247324914906
  (2, 131148)	0.26813732400121504
  (2, 517020)	0.10231390424363242
  (2, 566733)	0.3536332852590365
  :	:
  (1279998, 151216)	0.1866769313524877
  (1279998, 226659)	0.21326620689238973
  (1279998, 240750)	0.22914088411240496
  (1279998, 562615)	0.2034240179077971
 

TRAINING THE LOGISTIC REGRESSION MODEL

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

ACCURACY SCORE

In [ ]:
#Score of training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy : ', training_data_accuracy)

Accuracy :  0.81171640625


# New Section

In [ ]:
#Score of test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy : ', test_data_accuracy)

Accuracy :  0.800215625


SAVING THE TRAINED MODEL

In [ ]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

USING THE SAVED MODEL

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

X_new = X_test[80]

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative')
else:
  print('Positive')

[1]
Positive


In [ ]:
def preprocess_text(text):
    port_stem = PorterStemmer()
    processed_text = re.sub('[^a-zA-Z]', ' ', text).lower()
    processed_text = processed_text.split()
    processed_text = [port_stem.stem(word) for word in processed_text if word not in stopwords.words('english')]
    return ' '.join(processed_text)

# Custom input
custom_text = "instagram is very informative!"


processed_custom_text = preprocess_text(custom_text)

custom_text_transformed = vectorizer.transform([processed_custom_text])

custom_prediction = model.predict(custom_text_transformed)

# Print the result
if custom_prediction[0] == 0:
    print("Negative.")
else:
    print("Positive.")


Positive.
